In [1]:
#samplecontract.py
from pyteal import *
from algosdk.v2client.algod import AlgodClient
from algosdk import account, mnemonic

from algosdk.future import transaction
import base64

In [2]:


"""Basic Counter Application"""

def approval_program():
    handle_creation = Seq(
        App.globalPut(Bytes("Count"), Int(0)),
        Return(Int(1))
    )
    handle_optin = Return(Int(0)) # Could also be Reject()

    handle_closeout = Return(Int(0))

    handle_updateapp = Return(Int(0))

    handle_deleteapp = Return(Int(0))

    # Declare the ScratchVar as a Python variable _outside_ the expression tree
    scratchCount = ScratchVar(TealType.uint64)
    
    add = Seq(
        # The initial `store` for the scratch var sets the value to 
        # whatever is in the `Count` global state variable
        scratchCount.store(App.globalGet(Bytes("Count"))), 
        # Increment the value stored in the scratch var 
        # and update the global state variable 
        App.globalPut(Bytes("Count"), scratchCount.load() + Int(1)),
        Return(Int(1))
    )

    deduct = Seq(
        # The initial `store` for the scratch var sets the value to 
        # whatever is in the `Count` global state variable
        scratchCount.store(App.globalGet(Bytes("Count"))),
        # Check if the value would be negative by decrementing 
        If(scratchCount.load() > Int(0),
            # If the value is > 0, decrement the value stored 
            # in the scratch var and update the global state variable
            App.globalPut(Bytes("Count"), scratchCount.load() - Int(1)),
        ),
        Return(Int(1))
    )
    handle_noop = Seq(
        # First, lets fail immediately if this transaction is grouped with any others
        Assert(Global.group_size() == Int(1)), 
        Cond(
            [Txn.application_args[0] == Bytes("Add"), add], 
            [Txn.application_args[0] == Bytes("Deduct"), deduct]
        )
    )

    program = Cond(
        [Txn.application_id() == Int(0), handle_creation],
        [Txn.on_completion() == OnComplete.OptIn, handle_optin],
        [Txn.on_completion() == OnComplete.CloseOut, handle_closeout],
        [Txn.on_completion() == OnComplete.UpdateApplication, handle_updateapp],
        [Txn.on_completion() == OnComplete.DeleteApplication, handle_deleteapp],
        [Txn.on_completion() == OnComplete.NoOp, handle_noop]
    )

    return compileTeal(program, Mode.Application, version=5)

def clear_state_program():
    program = Return(Int(1))
    return compileTeal(program, Mode.Application, version=5)

    


In [3]:
# helper function to compile program source
def compile_program(client, source_code):
    compile_response = client.compile(source_code)
    return base64.b64decode(compile_response['result'])

# helper function that converts a mnemonic passphrase into a private signing key
def get_private_key_from_mnemonic(mn) :
    private_key = mnemonic.to_private_key(mn)
    return private_key

# helper function that formats global state for printing
def format_state(state):
    formatted = {}
    for item in state:
        key = item['key']
        value = item['value']
        formatted_key = base64.b64decode(key).decode('utf-8')
        if value['type'] == 1:
            # byte string
            if formatted_key == 'voted':
                formatted_value = base64.b64decode(value['bytes']).decode('utf-8')
            else:
                formatted_value = value['bytes']
            formatted[formatted_key] = formatted_value
        else:
            # integer
            formatted[formatted_key] = value['uint']
    return formatted

# helper function to read app global state
def read_global_state(client, app_id):
    app = client.application_info(app_id)
    global_state = app['params']['global-state'] if "global-state" in app['params'] else []
    return format_state(global_state)

# create new application
def create_app(client, private_key, approval_program, clear_program, global_schema, local_schema):
    # define sender as creator
    sender = account.address_from_private_key(private_key)

    # declare on_complete as NoOp
    on_complete = transaction.OnComplete.NoOpOC.real

    # get node suggested parameters
    params = client.suggested_params()

    # create unsigned transaction
    txn = transaction.ApplicationCreateTxn(sender, params, on_complete, \
                                            approval_program, clear_program, \
                                            global_schema, local_schema)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # wait for confirmation
    try:
        transaction_response = transaction.wait_for_confirmation(client, tx_id, 4)
        print("TXID: ", tx_id)
        print("Result confirmed in round: {}".format(transaction_response['confirmed-round']))

    except Exception as err:
        print(err)
        return

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    app_id = transaction_response['application-index']
    print("Created new app-id:", app_id)

    return app_id

In [4]:
# user declared account mnemonics
creator_mnemonic = "joke depend gain feature vintage spring permit loyal need crouch leopard cake major fit nurse only focus s\
catter indicate weasel door subject typical abandon wave"
# user declared algod connection parameters.
# Node must have EnableDeveloperAPI set to true in its config
algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

In [5]:

# initialize an algodClient
algod_client = AlgodClient(algod_token, algod_address)
# define private keys
creator_private_key = get_private_key_from_mnemonic(creator_mnemonic)

# declare application state storage (immutable)
local_ints = 0
local_bytes = 0
global_ints = 1
global_bytes = 0
global_schema = transaction.StateSchema(global_ints, global_bytes)
local_schema = transaction.StateSchema(local_ints, local_bytes)

# compile program to TEAL assembly
with open("./approval.teal", "w") as f:
    approval_program_teal = approval_program()
    f.write(approval_program_teal)

# compile program to TEAL assembly
with open("./clear.teal", "w") as f:
    clear_state_program_teal = clear_state_program()
    f.write(clear_state_program_teal)

# compile program to binary
approval_program_compiled = compile_program(algod_client, approval_program_teal)

# compile program to binary
clear_state_program_compiled = compile_program(algod_client, clear_state_program_teal)

print("--------------------------------------------")
print("Deploying Counter application......")

# create new application
app_id = create_app(algod_client, creator_private_key, approval_program_compiled, clear_state_program_compiled, global_schema, local_schema)

# read global state of application
print("Global state:", read_global_state(algod_client, app_id))

--------------------------------------------
Deploying Counter application......
TXID:  JVHF4VOQ2AD7OR4RJIPBJ2LBMOUFVBV5DB6HQ3JYSNIIBOOZOCFA
Result confirmed in round: 5
Created new app-id: 5
Global state: {'Count': 0}


In [6]:
# call application
def call_app(client, private_key, index, app_args) :
    # declare sender
    sender = account.address_from_private_key(private_key)

    # get node suggested parameters
    params = client.suggested_params()

    # create unsigned transaction
    txn = transaction.ApplicationNoOpTxn(sender, params, index, app_args)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # wait for confirmation
    try:
        transaction_response = transaction.wait_for_confirmation(client, tx_id, 5)
        print("TXID: ", tx_id)
        print("Result confirmed in round: {}".format(transaction_response['confirmed-round']))

    except Exception as err:
        print(err)
        return
    print("Application called")

In [8]:
print("--------------------------------------------")
print("Calling Counter application......")
app_args = ["Add"]
call_app(algod_client, creator_private_key, app_id, app_args)

# read global state of application
print("Global state:", read_global_state(algod_client, app_id))

--------------------------------------------
Calling Counter application......
TXID:  SG3F356I5UX4KUR2BOFZBKGRWKGBLK7EFTVIV2AUTLFE5J4PS6GQ
Result confirmed in round: 7
Application called
Global state: {'Count': 2}
